In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_eager_execution()

from sklearn.datasets import make_moons
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [64]:
# create input function
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth',
                   'PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versocolor','Virginica']

def load_data():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1],TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1],TEST_URL)
    
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    X_train, y_train = train,train.pop('Species')
    
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    X_test, y_test = test,test.pop('Species')
    
    return (X_train,y_train), (X_test,y_test)

(X_train,y_train), (X_test,y_test) = load_data()
print (X_train)

     SepalLength  SepalWidth  PetalLength  PetalWidth
0            6.4         2.8          5.6         2.2
1            5.0         2.3          3.3         1.0
2            4.9         2.5          4.5         1.7
3            4.9         3.1          1.5         0.1
4            5.7         3.8          1.7         0.3
..           ...         ...          ...         ...
115          5.5         2.6          4.4         1.2
116          5.7         3.0          4.2         1.2
117          4.4         2.9          1.4         0.2
118          4.8         3.0          1.4         0.1
119          5.5         2.4          3.7         1.0

[120 rows x 4 columns]


In [65]:
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    return dataset.shuffle(1000,seed=42).repeat().batch(batch_size)



In [66]:
def eval_input_fn(features,labels,batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features,labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    return dataset.batch(batch_size)
    

In [67]:
feature_columns = []
for key in X_train.columns:
    feature_columns.append(tf.feature_column.numeric_column(key=key))

feature_columns


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [68]:
config = tf.estimator.RunConfig(tf_random_seed=42)

classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                       hidden_units=[10,10],
                                       n_classes=3,
                                       config=config)

batch_size = 100
train_step = 1000

classifier.train(
    input_fn=lambda:train_input_fn(X_train,y_train,batch_size),
    steps=train_step)


eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(X_test,y_test,batch_size))

print('eval_result',eval_result)
print('eval_result',eval_result['accuracy'])



INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpu8m1qx36', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028445A5D448>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run 

In [72]:

X_pred = {'SepalLength': [5.1,5.9,6.9],
          'SepalWidth': [3.3,3.0,3.1],
          'PetalLength': [1.7,4.2,5.4],
          'PetalWidth': [0.5,1.5,2.1]}

expected = ['Setosa','Versocolor','Virginica']
predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(X_pred,
                                 labels = None,
                                 batch_size = batch_size))

template = ( 'Prediction is "{}" ({:.1f}%), expected "{}"')
            
for pred_dict,label in zip(predictions,expected):
    class_id = pred_dict['class_ids'][0]
    prob = pred_dict['probabilities'][class_id]
    
    print(template.format(SPECIES[class_id],
                         100 * prob , label))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


Exception ignored in: <generator object Estimator.predict at 0x000002844878D6C8>
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\estimator.py", line 650, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "C:\Users\user\Anaconda3\lib\contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 5483, in get_controller
    yield g
  File "C:\Users\user\Anaconda3\lib\contextlib.py", line 130, in __exit__
    self.gen.throw(type, value, traceback)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py", line 5298, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\user\AppData\Local\Temp\tmpu8m1qx36\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (99.8%), expected "Setosa"
Prediction is "Versocolor" (99.8%), expected "Versocolor"
Prediction is "Virginica" (92.2%), expected "Virginica"


## What does batch, repeat, and shuffle do with TensorFlow Dataset
dataset.shuffle(buffer_size=3) will allocate a buffer of size 3 for picking random entries. This buffer will be connected to the source dataset. We could image it like this:
Let's assume that the entry 2 was taken from the random buffer. Free space is filled by the next element from the source buffer, that is 4:
We continue reading till nothing is left:

[1,2,3] <= [4,5,6]

2 <= [1,3,4] <= [5,6]

1 <= [3,4,5] <= [6]

5 <= [3,4,6] <= []

3 <= [4,6]   <= []

6 <= [4]      <= []

4 <= []      <= []

As soon as all the entries are read from the dataset and you try to read the next element, the dataset will throw an error. That's where ds.repeat() comes into play. It will re-initialize the dataset, making it again like this:

[1,2,3] <= [4,5,6]

What will ds.batch() produce

The ds.batch() will take first batch_size entries and make a batch out of them. So, batch size of 3 for our example dataset will produce two batch records:

[2,1,5]
[3,6,4]